<h1> <center> Table of Contents </center> </h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#1.-Importing-Required-Libraries-and-Data"> Importing Required Libraries and Data</a></li>
        <li><a href="#2.-Data-Pre-processing-and-Preparation"> Data Pre-processing and Preparation</a></li>
        <li><a href="#3.-Different-Methods">Different Methods</a> </li>
        <li><a href="#4.-Summary-of-Different-Methods">Summary of Different Methods</a></li>                 
      </li>
    </ol>
</div>

# 1. Importing Required Libraries and Data

In [1]:
# pip installs
# pip install python-Levenshtein
# pip install "fuzzywuzzy==0.18.0"
# pip install rapidfuzz
# pip install spacy
# pip install gensim
# !pip install python-Levenshtein
# from rank_bm25 import BM25Okapi
# nltk.download('punkt')
# nltk.download('stopwords') 
# nltk.download('wordnet')
# !pip install rank_bm25
# !python -m spacy download en_core_web_lg 
# !pip install fast-autocomplete

from gensim.utils import simple_preprocess
import spacy
from tqdm import tqdm

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import rapidfuzz
from rapidfuzz import process, utils

import re
from re import sub

from nltk.tokenize import word_tokenize
import nltk

from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm

import pandas as pd
from fast_autocomplete import AutoComplete

from pandas.core.common import flatten
import timeit

In [2]:
#importing the file

dfdashboards = pd.read_csv('microstrategy_and_dashboard.csv')
dfmetrics = pd.read_csv('tableau_metric.csv')
dfdashboards.head()

,id,dashboards,caption,datasource,url,Global_usage,users_id_1,user_id_2
0,139444,Ad Analysis,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,1717,532,1185
1,139443,Ad Recap,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,974,523,451
2,139434,Aggregate Sales per Pt vs. CWD,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0|IRI Agg 34.0,https:andanddashboards.armadilloinsight.comand...,851,422,429
3,139466,Brand Control,Brand Family|Brand Levels|Brand|Main Brand Fam...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,923,625,298
4,139465,Brand In and Out Scorecard,Segment|Time as Date,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,876,520,356


In [3]:
#converting column values to lowercase

dfdashboards = dfdashboards.apply(lambda x: x.astype(str).str.lower())
dfmetrics = dfmetrics.apply(lambda x: x.astype(str).str.lower())

In [4]:
#converting global usage to numeric

dfdashboards.Global_usage = pd.to_numeric(dfdashboards.Global_usage)

In [5]:
#splitting caption and descriptors columns 

dfdashboards = dfdashboards[['dashboards','caption','Global_usage']]
dfdashboards['metric_names'] = dfdashboards.caption.str.split('|')
dfmetrics['descriptors'] = dfmetrics.descriptors.str.split('|')

In [6]:
#converting metric names and descriptors in each row values to multiple rows

temp1 = dfdashboards.set_index(['dashboards'])['metric_names'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'caption'})
temp2 = dfmetrics.set_index(['metric_name'])['descriptors'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'descriptor'})

In [7]:
#merging metrics and descriptors

master = temp1.merge(temp2, how = 'inner', left_on ='caption', right_on = 'metric_name')[['dashboards','metric_name','descriptor']]
master = master.rename(columns={"dashboards":"dashboard_names", "metric_name": "metric_names", "descriptor": "descriptor_names"})
master

,dashboard_names,metric_names,descriptor_names
0,ad analysis,beer style level value,macro style value
1,ad analysis,beer style level value,mezzo style value
2,ad analysis,beer style level value,micro style valu
3,ad analysis,beer style level value,beer style value
4,ad analysis,beer style level value,altbier
...,...,...,...
236553,surveyed stores,surveyed stores,2018
236554,surveyed stores,surveyed stores,2019
236555,surveyed stores,surveyed stores,2020
236556,surveyed stores,surveyed stores,2021


In [8]:
#checking for metric and descriptors

master[(master['dashboard_names'] == 'volume rank comp') | (master['descriptor_names'] == 'volume rank comp') | (master['metric_names'] == 'volume rank comp')].head()

,dashboard_names,metric_names,descriptor_names
3060,volume rank comp,beer style level value,macro style value
3061,volume rank comp,beer style level value,mezzo style value
3062,volume rank comp,beer style level value,micro style valu
3063,volume rank comp,beer style level value,beer style value
3064,volume rank comp,beer style level value,altbier


### Creating Corpus

In [9]:
#creating corpus

dashboard_names_list = list(master.dashboard_names.unique())
metric_names_list = list(master.metric_names.unique())
descriptor_names_list = list(master.descriptor_names.unique())
mastercorpus = dashboard_names_list + metric_names_list + descriptor_names_list

In [10]:
#pre-processing corpus

mastercorpus = list(filter(None,mastercorpus))
mastercorpus

['ad analysis',
 'ad recap',
 'aggregate sales per pt vs. cwd',
 'brands on ad',
 'brewery comparisons',
 'company ',
 'company brands over time',
 'r geography',
 'r geography over time',
 'company',
 'category and segment analysis',
 'competitive set',
 'dimensions over time',
 'dollar rank comp',
 'dollar ranker with cwd',
 'dollar vs. ad share',
 'dollars vs. growth',
 'feature ad count',
 'footprint',
 'line geogs over time',
 'merch efficiency',
 'merch efficiency c&amp;r',
 'merch execution',
 'merch ranker',
 'merch vs. non-merch',
 'our brands',
 'our brands dollar ranker',
 'package analysis',
 'price analysis',
 'price bucket analysis',
 'quick scorecard',
 'rankers',
 'sales per pt vs. cwd',
 'style analysis',
 'topline',
 'velocity',
 'volume rank comp',
 'market index analysis',
 'market index analysis visual',
 'price promotion count',
 'the humbler',
 'brand control',
 'item audit',
 'market share',
 'our packages',
 'depletions month cube with inventory and shipmentmen

# 2. Data Pre-processing and Preparation

- lower case (done)
- lemmatization

## Search suggestions flow :
1. Suggestion (includes dashboard title, metric names and descriptors)
- Extract dashboard names list  
- Final suggestions 

## Efficient approaches:

- storing metric and descriptors as keys, and dashboards names as values in a dictionary 
- weightage to dashboard names, followed by metric and descriptors

##  Search Engine using :

- Regex expressions
- Fuzzyuwuzzy string matching
- Global Usage
- Auto-Complete
- Wordnet


# 3. Different Methods

## 3.1 Method 1: RegEx Regular Expressions (Baseline Model)

<b>Description</b>: A RegEx, or Regular Expression, is a sequence of characters that forms a search pattern. RegEx can be used to check if a string contains the specified search pattern. 
- The <b>re</b> module offers a set of functions that allows us to search a string for a match.
- The <b>search</b> Returns a Match object if there is a match anywhere in the string.

In [11]:
#regex string matching

def regex_suggestions(input_query, corpus):
    return [x for x in corpus if re.search(input_query.lower(),x)]

In [12]:
def dashboard_names_suggestion(master, suggestions):
   
    dashboard_names = []
       
    for i in suggestions:
       
        if i in dashboard_names_list:
            
            dashboard_names.append(i)
        
        elif i in metric_names_list:
            
            dashboard_names = dashboard_names + (master.loc[(master.metric_names.str.lower() == i)].dashboard_names).to_list()
        
        else:
            
            dashboard_names = dashboard_names + (master.loc[(master.descriptor_names.str.lower() == i)].dashboard_names).to_list()

    return dashboard_names


In [13]:
#regex input query

inputQuery = 'volume'
regexSuggestionsList = regex_suggestions(inputQuery, mastercorpus)
print(regexSuggestionsList)

#regex search output

regexDashboardsList = list(dashboard_names_suggestion(master, regexSuggestionsList))

regexSearchOutput = []
regexSearchOutput = list(dict.fromkeys(regexDashboardsList))
regexSearchOutput[:10]

['volume rank comp', 'rank (volume)', 'volume sales', 'volume sales change ya', 'volume sales per point', 'volume sales per percent last year', 'volume sales per pt change ya', 'volume sale']


['volume rank comp',
 'quick scorecard',
 'ad analysis',
 'r geography over time',
 'category and segment analysis',
 'competitive set',
 'dimensions over time',
 'line geogs over time',
 'market share',
 'package analysis']

## 3.2 Method 2: fuzzywuzzy String Matching

FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern. Basically it uses <b>Levenshtein Distance</b> to calculate the differences between sequences.

In [33]:
def fuzzywuzzy_suggestion(corpus, input_query):
   
    for i in [corpus]:
        
        return process.extract(input_query, i, limit=10)

In [34]:
#Input query:

InputQuery = 'Price per unit '

fuzzywuzzySuggestions = fuzzywuzzy_suggestion(mastercorpus,InputQuery)

start = timeit.default_timer()
fuzzywuzzySuggestionsList = []
for i in fuzzywuzzySuggestions:
    fuzzywuzzySuggestionsList.append(i[0])

fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(master, fuzzywuzzySuggestionsList))

fuzzywuzzySearchOutput = []
fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
fuzzywuzzySearchOutput[:10]

['dollar rank comp',
 'dollar ranker with cwd',
 'price analysis',
 'price bucket analysis',
 'rankers',
 'velocity',
 'depletions month cube with inventory and shipmentments',
 'depletionsletions cube',
 'retail month cube',
 'retail cube to  brand quality level']

In [16]:
# start_time = datetime.now()
# suggestion = fuzzywuzzy_suggestion(text,'rever')
# print(suggestion)

# dashboard_suggestions = list(dashboard_names_suggestion(df_final,suggestion))

# search_output = []
# search_output = list(dict.fromkeys(dashboard_suggestions))
# search_output
# end_time = datetime.now()
# print('Duration: {}'.format(end_time - start_time))

## 3.3 Method 3: Global Usage

The Global Usage is provided in the dataset. By sorting it, top 10 dashboard names is being generated. 

In [17]:
def popular_dashboard_suggestion(dfdashboards, globalUsageDashboardList): 
    
    return dfdashboards[dfdashboards['dashboards'].isin(globalUsageDashboardList)].sort_values(by = 'Global_usage',ascending = False)['dashboards'].tolist()            

In [18]:
#popular dashboard suggestions
 
globalUsageSeachOutput = popular_dashboard_suggestion(dfdashboards,fuzzywuzzySearchOutput)
globalUsageSeachOutput[:10]

['dollar ranker with cwd',
 'category and segment analysis',
 'dollar rank comp',
 'style analysis',
 'rankers',
 'price analysis',
 'ad analysis',
 'sales team opportunitshis year ',
 'r geography over time',
 'depletions month last year by distributor ']

In [19]:
#checking for metric and descriptors

df_dashboard_suggestions = master[(master['dashboard_names'] == '') | (master['descriptor_names'] == '214 west main st') | (master['metric_names'] == '')]
df_dashboard_suggestions.dashboard_names.nunique()

15

## 3.4 Method 4: Fast-AutoComplete

Fast autocomplete using <b>Directed Word Graph (DWG)</b> and <b>Levenshtein Edit Distance.</b>

In [20]:
#corpus dictionary creations

dashboard_dict = dict(enumerate(master.dashboard_names.unique().flatten(), 1))
dashboard_dict = {v: k  for k,v in dashboard_dict.items()}
dashboard_dict

metric_dict = dict(enumerate(master.metric_names.unique().flatten(), 1))
metric_dict = {v: k  for k,v in metric_dict.items()}
metric_dict

descriptor_dict = dict(enumerate(master.descriptor_names.unique().flatten(), 1))
descriptor_dict = {v: k  for k,v in descriptor_dict.items()}
descriptor_dict

masterDictionary = {}
masterDictionary.update(dashboard_dict)
masterDictionary.update(metric_dict)
masterDictionary.update(descriptor_dict)


for k,v in masterDictionary.items():
    masterDictionary[k] = {}
    

autocomplete = AutoComplete(words = masterDictionary)

#metric_dict = df_final.caption.unique().to_dict()
#descriptor_dict = df_final.descriptor.unique().to_dict()

In [21]:
#input to autocomplete

input_query = 'volumne'

autoCompleteSuggetionsList = autocomplete.search(word = input_query, max_cost= 3, size = 100)
autoCompleteSuggetionsList = list(flatten(autoCompleteSuggetionsList))

#autossuggestions

autoCompleteDashboardsList = dashboard_names_suggestion(master,autoCompleteSuggetionsList)

autoCompleteSearchOutput = []
autoCompleteSearchOutput = list(dict.fromkeys(autoCompleteDashboardsList))
autoCompleteSearchOutput[:10]

['ad analysis',
 'aggregate sales per pt vs. cwd',
 'brands on ad',
 'brewery comparisons',
 'company ',
 'company brands over time',
 'r geography',
 'r geography over time',
 'company',
 'category and segment analysis']

In [22]:
autoCompleteSuggetionsList

['volume sale',
 'volume sales',
 'volume rank comp',
 'volume sales change ya',
 'volume sales per point',
 'volume sales per pt change ya',
 'volume sales per percent last year']

# 4. Summary of Different Methods

In [25]:
def finalSuggestion(input_query):
        
        #regex
        start = timeit.default_timer()
        regexSuggestionsList = regex_suggestions(input_query, mastercorpus)
        regexDashboardsList = list(dashboard_names_suggestion(master, regexSuggestionsList))
        regexSearchOutput = []
        regexSearchOutput = list(dict.fromkeys(regexDashboardsList))
        stop = timeit.default_timer()
        execution_time = stop - start
        print("Time taken by Method 1: RegEx "+str(execution_time))
        
        #fuzzywuzzy
        
        start = timeit.default_timer()
        fuzzywuzzySuggestions = fuzzywuzzy_suggestion(mastercorpus,input_query)
        fuzzywuzzySuggestionsList = []
        for i in fuzzywuzzySuggestions:
            fuzzywuzzySuggestionsList.append(i[0])
        fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(master, fuzzywuzzySuggestionsList))
        fuzzywuzzySearchOutput = []
        fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
        stop = timeit.default_timer()
        execution_time = stop - start
        print("Time taken by Method 2: fuzzywuzzy String Matching "+str(execution_time))
        
        
        #using Global Usage
        
        start = timeit.default_timer()
        globalUsageSearchOutput = popular_dashboard_suggestion(dfdashboards,fuzzywuzzySearchOutput)
        stop = timeit.default_timer()
        execution_time = stop - start
        print("Time taken by Method 3: fuzzywuzzy with Global Usage "+str(execution_time))

        #autossuggestions

        start = timeit.default_timer()
        autoCompleteSuggetionsList = autocomplete.search(word = input_query, max_cost= 3, size = 100) 
        autoCompleteSuggetionsList = list(flatten(autoCompleteSuggetionsList))
        autoCompleteDashboardsList = dashboard_names_suggestion(master,autoCompleteSuggetionsList)
        autoCompleteSearchOutput = []
        autoCompleteSearchOutput = list(dict.fromkeys(autoCompleteDashboardsList))
        stop = timeit.default_timer()
        execution_time = stop - start
        print("Time taken by Method 4: fuzzywuzzy with AutoComplete "+str(execution_time))
        
        return regexSearchOutput[:10],fuzzywuzzySearchOutput[:10],globalUsageSearchOutput[:10],autoCompleteSearchOutput[:10]

In [26]:
searchOutputs = finalSuggestion('2 silos brewing co hi wire brand family craft craftesque')

column_names = ['RegEx', 'fuzzywuzzy String Matching', 'fuzzywuzzy with Global Usage','fuzzywuzzy with AutoComplete']
finalSuggestionDF = pd.DataFrame(searchOutputs).transpose().set_axis(column_names, axis =1)
finalSuggestionDF

Time taken by Method 1: RegEx 0.017958391999997048
Time taken by Method 2: fuzzywuzzy String Matching 1.0545900109999877
Time taken by Method 3: fuzzywuzzy with Global Usage 0.0009385500000007596
Time taken by Method 4: fuzzywuzzy with AutoComplete 0.1211298759999977


,RegEx,fuzzywuzzy String Matching,fuzzywuzzy with Global Usage,fuzzywuzzy with AutoComplete
0,None,ad analysis,dollar ranker with cwd,brand control
1,None,ad recap,category and segment analysis,None
2,None,aggregate sales per pt vs. cwd,dollar rank comp,None
3,None,brand control,style analysis,None
4,None,brewery comparisons,rankers,None
5,None,company,price analysis,None
6,None,company brands over time,market index analysis,None
7,None,r geography,ad analysis,None
8,None,r geography over time,brewery comparisons,None
9,None,company,distribution gains and losses cube vs. last month,None


In [ ]:
# def final_suggestion(input_query, version):
   
#     if version == 'FuzzyWuzzy':
        
#         suggestion = fuzzywuzzy_suggestion(text,'3361')
#         dashboard_suggestions = list(dashboard_names_suggestion(df_final,suggestion))

#         search_output = []
#         search_output = list(dict.fromkeys(dashboard_suggestions))
#         return search_output
    
#     else version == 'regex'
#         return regex_suggestions('sales', text)
    
#     else version == 'fuzzyWith'


## Versions:

    1.Regex expression
    2.Fuzzywuzzy
    3.Fuzzywuzzy + Global_Usage
    5.AutoComplete + FuzzyWuzzy
    6.Wordnet + FuzzyWuzzy
    7.AutoComplete + Wordnet + FuzzyWuzzy + Global_Usage
    8.ElasticSearch 
    9.Doc2vec

## Search query conditions ??

- As the user is more concerned about the dashboard title, the search query should first retrieve the most appropriate dashboard name

- If user searches with descriptors or metric then the corresponding dashboard names should be suggested along with metric

- Dictionary with dashboard names as keys,metric_names,descriptors as values & displaying corresponding keys for value in search suggestion

- output string: dashboard name, metric

# Using TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

search_terms = 'dollar sales'
documents = text

doc_vectors = TfidfVectorizer().fit_transform([search_terms] + documents)

cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
document_scores = [item.item() for item in cosine_similarities[1:]]

tuples = list(zip(text,document_scores))
df_scores = pd.DataFrame(tuples, columns = ['text','document_scores'])
df_scores.sort_values('document_scores', ascending =False).head(10)

In [ ]:
doc_vectors.nonzero()

# Using Glove's Model

In [ ]:
from re import sub
from gensim.utils import simple_preprocess

query_string = 'area'
documents = text

stopwords = ['the', 'and', 'are', 'a']

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)




# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


In [ ]:
# import gensim.downloader as api
# from gensim.corpora import Dictionary
# from gensim.models import TfidfModel
# from gensim.models import WordEmbeddingSimilarityIndex
# from gensim.similarities import SparseTermSimilarityMatrix
# from gensim.similarities import SoftCosineSimilarity

# # Load the model: this is a big file, can take a while to download and open
# glove = api.load("glove-wiki-gigaword-50")    
# similarity_index = WordEmbeddingSimilarityIndex(glove)



In [ ]:
# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


# Docsim

## Other Word Embedding techniques:
BERT, BM25,fasttext, FAISS, SEQ2SEQ, WORD2VEC,NMSLIB

In [ ]:
from IPython.display import Image
Image(filename=r'D:\internship Experience\Fintech 2021\Different_techniques.png', width = 500, height =100)